Transfer learning using cryptobert and roberta

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import pandas as pd
from scipy.special import softmax

/Users/endy/.pyenv/versions/3.10.6/envs/crypto/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install torch

In [3]:
!pip install protobuf==3.20.0

## Step 1 - Importing a sample of some Bitcoin Tweet Data from Kaggle to begin analysing the model

In [4]:
data = pd.read_csv('~/code/giadapi/crypto/data/raw/bitcoin_tweets1000000.csv', nrows = 1000)

# "\\wsl.localhost\Ubuntu\home\peter\code\giadapi\crypto\kaggle-tweets.zip"

In [5]:
data

,Unnamed: 0,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet,cleanText,Polarity Score,sentiment
0,0,DeSota Wilson,"Atlanta, GA","Biz Consultant, real estate, fintech, startups...",2009-04-26 20:05:09,8534.0,7605,4838,False,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,['bitcoin'],Twitter Web App,False,blue ridge bank shares halted nyse bitcoin atm...,0.00,0
1,1,CryptoND,NaN,😎 BITCOINLIVE is a Dutch platform aimed at inf...,2019-10-17 20:12:10,6769.0,1532,25483,False,2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...","['Thursday', 'Btc', 'wallet', 'security']",Twitter for Android,False,today thursday take friend leowandersleb btc w...,0.00,0
2,2,Tdlmatias,"London, England","IM Academy : The best #forex, #SelfEducation, ...",2014-11-10 10:50:37,128.0,332,924,False,2021-02-10 23:54:48,"Guys evening, I have read this article about B...",NaN,Twitter Web App,False,guys evening read article btc would like share,0.00,0
3,3,Crypto is the future,NaN,I will post a lot of buying signals for BTC tr...,2019-09-28 16:48:12,625.0,129,14,False,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,"['Bitcoin', 'FX', 'BTC', 'crypto']",dlvr.it,False,btc big chance billion price bitcoin btc crypto,0.00,0
4,4,Alex Kirchmaier 🇦🇹🇸🇪 #FactsSuperspreader,Europa,Co-founder @RENJERJerky | Forbes 30Under30 | I...,2016-02-03 13:15:55,1249.0,1472,10482,False,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,['BTC'],Twitter Web App,False,network secured nodes today soon biggest bears...,-0.25,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,Szymon Kostrzewa,NaN,NaN,2016-04-02 12:33:50,7.0,75,31,False,2021-02-10 17:27:33,🤝 Follow me on @betfury_io. Let's hunt for Bit...,NaN,Twitter Web App,False,follow betfury_io let hunt bitcoins together g...,0.30,1
996,996,Crypto Zombie,New York,🎥 YouTube: Crypto Zombie 🧟‍♂️ #Bitcoin and #bl...,2010-01-18 23:52:55,59416.0,1058,23385,False,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,['Bitcoin'],Twitter Web App,False,bitcoin falling tesla pump top btc worried dip...,0.50,1
997,997,Brian Harrington,"Anaheim, CA",#Bitcoin is actively replacing the dollar. Can...,2009-08-19 20:21:01,7687.0,2322,83966,False,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,NaN,Twitter Web App,False,within driving distance iowa city definitely c...,0.00,0
998,998,🔸️Bitcoin Pixel🔸️,NaN,I discovered crypto in 2019 on Youtube. Since ...,2015-07-08 13:44:42,1873.0,75,676,False,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,"['BTC', 'ETH', 'DOT']",Twitter for Android,False,question please finally sold avax amp cake bou...,0.00,0


In [6]:
data = data[['text', 'date']]

In [7]:
data['text'][0]

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement https://t.co/xaaZmaJKiV @MyBlueRidgeBank… https://t.co/sgBxMkP1SI'

## Step 2 - Cleaning the data

In [8]:
# Preprocess text (username and link placeholders) - this is suggested from the example in 
# https://huggingface.co/cardiffnlp/twitter-xlm-roberta-base-sentiment
def preprocess_1(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

preprocess_1(data['text'][0])

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement http @user http'

In [9]:
# I have changed this to remove more information

def preprocess_2(text):
    new_text = []
    for t in text.split(" "):
        t = '' if t.startswith('@') and len(t) > 1 else t
        t = '' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

sample_test = preprocess_2(data['text'][0])
sample_test

'Blue Ridge Bank shares halted by NYSE after #bitcoin ATM announcement   '

In [10]:
#use the preprocess_2 to clean the data

data['process_text'] = data.text
data['label'] = data.text
data['score'] = data.text
for i in range(len(data)):
    data['process_text'][i] = preprocess_2(data['text'][i])

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72124/3797914433.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['process_text'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72124/3797914433.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data.text
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72124/3797914433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

## Step 3 - analyse the language and sentiments by pretrained model

In [11]:
!pyenv local crypto

In [12]:
#test the twitter xlm

from transformers import pipeline
model_path = "ElKulako/cryptobert"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
test_sent = sentiment_task(sample_test)

In [13]:
test_sent[0]['score']

0.9938846230506897

In [14]:
test_sent[0]['label']

'Neutral'

In [15]:
for i in range(len(data)):
    sentiment = sentiment_task(data['process_text'][i])
    data['label'][i] = sentiment[0]['label']
    data['score'][i] = sentiment[0]['score']

/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72124/2277679823.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'][i] = sentiment[0]['label']
/var/folders/d5/bxgljmg10q70y4xmy233dhfr0000gn/T/ipykernel_72124/2277679823.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['score'][i] = sentiment[0]['score']


In [16]:
data

,text,date,process_text,label,score
0,Blue Ridge Bank shares halted by NYSE after #b...,2021-02-10 23:59:04,Blue Ridge Bank shares halted by NYSE after #b...,Neutral,0.993885
1,"😎 Today, that's this #Thursday, we will do a ""...",2021-02-10 23:58:48,"😎 Today, that's this #Thursday, we will do a ""...",Neutral,0.849627
2,"Guys evening, I have read this article about B...",2021-02-10 23:54:48,"Guys evening, I have read this article about B...",Bearish,0.780925
3,$BTC A big chance in a billion! Price: \487264...,2021-02-10 23:54:33,$BTC A big chance in a billion! Price: \487264...,Bullish,0.653737
4,This network is secured by 9 508 nodes as of t...,2021-02-10 23:54:06,This network is secured by 9 508 nodes as of t...,Bullish,0.907657
...,...,...,...,...,...
995,🤝 Follow me on @betfury_io. Let's hunt for Bit...,2021-02-10 17:27:33,🤝 Follow me on Let's hunt for Bitcoins togeth...,Neutral,0.62777
996,#Bitcoin falling after Tesla pump! Was that th...,2021-02-10 17:27:16,#Bitcoin falling after Tesla pump! Was that th...,Bullish,0.630188
997,If you're within driving distance of Iowa City...,2021-02-10 17:27:13,If you're within driving distance of Iowa City...,Neutral,0.616015
998,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,2021-02-10 17:27:10,🔴 QUESTION PLEASE 🔴\n\nFinally I sold $AVAX &a...,Neutral,0.617814


## Step 4 - Compare the model performance

In [56]:
data.to_csv('xlm.csv')